# NTOU Machine learning HW1
## by 00857005 周固廷

In this exercise, we try to compare the accuracy of the two classifiers: k-NN and linear SVM on  the Breast Cancer Wisconsin (Diagnostic) dataset by nested cross-validation.  The following are the settings for this exercise.

1. Hyperparameters:
    The hyperparameter for k-NN is n_neighbors and the candidates for this hyperparameter are [1,3,5,7].
    The hyperparameter for linear SVM is C and the candidates for C are [0.01,0.1,1,10].
2. Nested cross-validation
   outer-loop: 10-fold stratified cross-validation
   inner-loop: GridSearchCV with cv=5
3. dataset
    import sklearn.datasets as ds
    data, target = ds.load_breast_cancer(True)

Useful links:
    + linear SVM (sklearn.svm.SVC): https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

    + k-nearest neighbor classifier (sklearn.neighbors.KNeighborsClassifier): https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

    + dataset: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer

In [1]:
import sklearn.datasets as ds
from sklearn.model_selection import StratifiedKFold,cross_val_score,GridSearchCV
import numpy as np

data, target = ds.load_breast_cancer(return_X_y = True)

# 1. SVM

In [2]:
#SVM
from sklearn import svm
svmClf = svm.SVC()
svmClf.fit(data, target)
svmClf.score(data, target)

0.9226713532513181

In [3]:
#cv代表cross validation
outer_kFold_svm = StratifiedKFold(n_splits=10 , shuffle=True ,random_state=2)  #外層的分層抽樣 (拆成10fold)
inner_cv_svm = GridSearchCV(svmClf, {'C':[0.01,0.1,1,10]}, cv=5)  # 內層的validation tuning (拆成5fold)
svm_scores = cross_val_score(inner_cv_svm, data, target, cv=outer_kFold_svm)

In [4]:
print(svm_scores)

[0.87719298 0.92982456 0.89473684 0.92982456 0.87719298 0.9122807
 0.96491228 0.98245614 0.92982456 0.92857143]


In [5]:
#計算AUC
svm_auc_scores = cross_val_score(inner_cv_svm, data, target, cv=outer_kFold_svm, scoring='roc_auc')
print(svm_auc_scores)
print("svm mean: {}".format(svm_auc_scores.mean()))
print("svm std: {}".format(svm_auc_scores.std()))

[0.98181818 0.94545455 0.97883598 0.98677249 0.93518519 1.
 0.99074074 0.99470899 0.98677249 0.99183673]
svm mean: 0.979212533498248
svm std: 0.020405330753281024


# 2. k-NN

In [6]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knnClf = KNeighborsClassifier()
knnClf.fit(data, target)
knnClf.score(data, target)

0.9472759226713533

In [7]:
#cv代表cross validation
outer_kFold_knn = StratifiedKFold(n_splits=10 , shuffle=True ,random_state=2)  #外層的分層抽樣 (拆成10fold)
inner_cv_knn = GridSearchCV(knnClf, {'n_neighbors':[1,3,5,7]}, cv=5)  # 內層的validation tuning (拆成5fold)
knn_scores = cross_val_score(inner_cv_knn, data, target, cv=outer_kFold_knn)

In [8]:
print(knn_scores)

[0.94736842 0.94736842 0.9122807  0.92982456 0.87719298 0.94736842
 0.94736842 0.98245614 0.92982456 0.92857143]


In [9]:
#計算AUC
knn_auc_scores = cross_val_score(inner_cv_knn, data, target, cv=outer_kFold_knn, scoring='roc_auc')
print(knn_auc_scores)
print("knn mean: {}".format(knn_auc_scores.mean()))
print("knn std: {}".format(knn_auc_scores.std()))

[0.96818182 0.99415584 0.91600529 0.96626984 0.88624339 0.99140212
 0.98082011 0.97222222 0.98941799 0.93741497]
knn mean: 0.960213358070501
knn std: 0.03404768174744221
